# Calculating Remote Sensing Indices

This notebook demonstrates how to calculate common remote sensing indices, such as NDVI (Normalized Difference Vegetation Index) and NDWI (Normalized Difference Water Index), using `rasterio` and `numpy` in Python. These indices are derived from multispectral raster data, such as Sentinel-2 imagery.

## Prerequisites
- Install required libraries: `rasterio`, `numpy`, `matplotlib` (listed in `requirements.txt`).
- A multi-band GeoTIFF file (e.g., `sample.tif` with Sentinel-2 bands). Replace the file path with your own raster file.

## Learning Objectives
- Calculate NDVI and NDWI from multispectral raster data.
- Visualize the resulting indices.
- Save the index rasters to new files.

In [ ]:
# Import required libraries
import rasterio
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Load the Raster File

Load a multi-band raster file (e.g., Sentinel-2 imagery). We assume the raster has at least the red (band 4) and near-infrared (NIR, band 8) bands for NDVI, and green (band 3) and NIR for NDWI.

In [ ]:
# Define the path to the raster file
raster_path = 'sample.tif'

# Open the raster file
with rasterio.open(raster_path) as src:
    # Read red (band 4), green (band 3), and NIR (band 8) for Sentinel-2
    red = src.read(4).astype(float)
    green = src.read(3).astype(float)
    nir = src.read(8).astype(float)
    profile = src.profile

# Print basic information
print(f'Raster shape: {red.shape}')
print(f'Number of bands: {profile["count"]}')

## Step 2: Calculate NDVI

NDVI is calculated as: `(NIR - Red) / (NIR + Red)`. This index highlights vegetation health, with values ranging from -1 to 1.

In [ ]:
# Calculate NDVI
ndvi = (nir - red) / (nir + red + 1e-10)  # Add small value to avoid division by zero

# Handle invalid values
ndvi = np.clip(ndvi, -1, 1)  # Ensure NDVI is within valid range

# Visualize NDVI
plt.figure(figsize=(8, 8))
plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar(label='NDVI')
plt.title('Normalized Difference Vegetation Index (NDVI)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 3: Calculate NDWI

NDWI is calculated as: `(Green - NIR) / (Green + NIR)`. This index highlights water bodies, with positive values indicating water.

In [ ]:
# Calculate NDWI
ndwi = (green - nir) / (green + nir + 1e-10)  # Add small value to avoid division by zero

# Handle invalid values
ndwi = np.clip(ndwi, -1, 1)  # Ensure NDWI is within valid range

# Visualize NDWI
plt.figure(figsize=(8, 8))
plt.imshow(ndwi, cmap='Blues', vmin=-1, vmax=1)
plt.colorbar(label='NDWI')
plt.title('Normalized Difference Water Index (NDWI)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 4: Save the Indices

Save the NDVI and NDWI rasters as new GeoTIFF files.

In [ ]:
# Update profile for single-band output
output_profile = profile.copy()
output_profile.update(count=1, dtype=rasterio.float32)

# Save NDVI
with rasterio.open('ndvi.tif', 'w', **output_profile) as dst:
    dst.write(ndvi.astype(rasterio.float32), 1)

# Save NDWI
with rasterio.open('ndwi.tif', 'w', **output_profile) as dst:
    dst.write(ndwi.astype(rasterio.float32), 1)

print('NDVI saved to: ndvi.tif')
print('NDWI saved to: ndwi.tif')

## Next Steps

- Replace `sample.tif` with your own multi-band raster (e.g., Sentinel-2 imagery).
- Adjust band indices based on your dataset (e.g., for Landsat, red and NIR bands differ).
- Try calculating other indices (e.g., EVI, SAVI) by modifying the formulas.
- Proceed to the next notebook (`05_basic_statistics_raster.ipynb`) to compute raster statistics.

## Notes
- Ensure the raster has the required bands (e.g., red, green, NIR for Sentinel-2).
- Check for invalid values (e.g., NaN) in the output indices.
- See `docs/installation.md` for troubleshooting library installation.